In [2]:
import pandas as pd
import os
import chardet
from lxml import etree
import requests
import pymysql

In [3]:
db = pymysql.connect(
    host='1.15.220.155',
    user='test',
    password='991125',
    database='spider',
    charset='utf8'
)

In [3]:
c = db.cursor()

c.execute('select link, isMovie from movies_universal where isMovie is not Null')
result = c.fetchall()

In [4]:
dataset = pd.DataFrame(columns=['url', 'target', 'data'])
for index, i in enumerate(result):
    dataset.loc[index, 'url'] = i[0]
    dataset.loc[index, 'target'] = i[1]
dataset

,url,target,data
0,http://www.zywlyy.cn/dy-xia-zai/ttdyxz10843.html,0,NaN
1,http://www.zywlyy.cn/dy-xia-zai/ttdyxz10778.html,0,NaN
2,http://www.zywlyy.cn/dy-xia-zai/ttdyxz10874.html,0,NaN
3,https://www.mofancy112.net/index.php/vod/detai...,1,NaN
4,https://www.mofancy112.net/index.php/vod/detai...,1,NaN
...,...,...,...
1432,http://www.mumuhd.com/voddetail/38537/,1,NaN
1433,http://www.wjcrystal.com/o/v667268.html,1,NaN
1434,https://www.6g2025.com/2025mv/2025176739.html,1,NaN
1435,https://www.6vhao.net/dy4/2022-03-06/42422.html,1,NaN


In [5]:
c = db.cursor()

c.execute('select domain from pms')
result = c.fetchall()

In [6]:
result = result[:600]

In [7]:
len_before = len(dataset)
for index, i in enumerate(result):
    dataset.loc[index+len_before, 'url'] = i[0]
    dataset.loc[index+len_before, 'target'] = 0

In [8]:
dataset

,url,target,data
0,http://www.zywlyy.cn/dy-xia-zai/ttdyxz10843.html,0,NaN
1,http://www.zywlyy.cn/dy-xia-zai/ttdyxz10778.html,0,NaN
2,http://www.zywlyy.cn/dy-xia-zai/ttdyxz10874.html,0,NaN
3,https://www.mofancy112.net/index.php/vod/detai...,1,NaN
4,https://www.mofancy112.net/index.php/vod/detai...,1,NaN
...,...,...,...
2032,www.hdpianyuan.com,0,NaN
2033,www.hdpianyuan.com,0,NaN
2034,www.hdshare.cn,0,NaN
2035,www.hdsky.net,0,NaN


In [6]:
def is_chinese(string):
    """
    检查整个字符串是否包含中文
    :param string: 需要检查的字符串
    :return: bool
    """
    for ch in string:
        if u'\u4e00' <= ch <= u'\u9fff':
            return True

    return False

In [10]:
import time
MAX_HREF_NUM = 1000

time_a = 0
time_b = time.time()
for i in range(len(dataset)):
    try:
        response = requests.get(dataset.loc[i, 'url'])
    except:
        print('time out.')
        continue
    results = response.content
    code = chardet.detect(results)
    try:
        results = results.decode(code['encoding'])
    except:
        try:
            results = results.decode('GB18030')
        except:
            try:
                results = results.decode('utf8', errors='ignore')
            except:
                print('file: {}, code: {}'.format(file, code))
                print('Not valid encode.')
                continue
    try:
        html = etree.HTML(results)
    except:
        continue
    if html is None:
        continue
#             html_data = html.xpath('//*[@href]/text()')
    html_data = html.xpath('//*[name(.)!="style" and name(.)!="script"]/text()')
    if len(html_data) >= MAX_HREF_NUM:  # 只选取前多少个带有链接的文本
        html_data = html_data[:MAX_HREF_NUM]
    content_data = html.xpath('//*[@content]/@content')
    content_data_valid = []
    for content in content_data:
        if is_chinese(content):
            content_data_valid.append(content)
    html_data += content_data_valid
    if html_data == []:
        continue
    one_item = ''
    code = ''
    for item in html_data:
        item = item.strip()
        one_item += ' '
        one_item += item
    dataset.loc[i, 'data'] = one_item
    if (i + 1) % 10 == 0:
        time_a = time_b
        time_b = time.time()
        print('{} records have been processed.time used:{:.2f}'.format(i, time_b - time_a))

9 records have been processed.time used:3.17
19 records have been processed.time used:3.95
29 records have been processed.time used:16.04
39 records have been processed.time used:14.27
49 records have been processed.time used:11.83
59 records have been processed.time used:13.97
69 records have been processed.time used:22.47
79 records have been processed.time used:6.97
89 records have been processed.time used:11.30
99 records have been processed.time used:3.59
109 records have been processed.time used:4.19
119 records have been processed.time used:3.11
129 records have been processed.time used:3.19
time out.
139 records have been processed.time used:6.36
149 records have been processed.time used:5.09
time out.
159 records have been processed.time used:13.04
169 records have been processed.time used:12.65
179 records have been processed.time used:12.24
time out.
189 records have been processed.time used:32.23
199 records have been processed.time used:9.08
209 records have been processed

In [11]:
dataset.to_csv('universal.csv')

In [4]:
dataset = pd.read_csv('universal.csv')

In [7]:
import time
MAX_HREF_NUM = 1000

time_a = 0
time_b = time.time()
for i in range(1437, len(dataset)):
    try:
        response = requests.get('http://' + dataset.loc[i, 'url'])
    except:
        print('time out.')
        continue
    results = response.content
    code = chardet.detect(results)
    try:
        results = results.decode(code['encoding'])
    except:
        try:
            results = results.decode('GB18030')
        except:
            try:
                results = results.decode('utf8', errors='ignore')
            except:
                print('file: {}, code: {}'.format(file, code))
                print('Not valid encode.')
                continue
    try:
        html = etree.HTML(results)
    except:
        continue
    if html is None:
        continue
#             html_data = html.xpath('//*[@href]/text()')
    html_data = html.xpath('//*[name(.)!="style" and name(.)!="script"]/text()')
    if len(html_data) >= MAX_HREF_NUM:  # 只选取前多少个带有链接的文本
        html_data = html_data[:MAX_HREF_NUM]
    content_data = html.xpath('//*[@content]/@content')
    content_data_valid = []
    for content in content_data:
        if is_chinese(content):
            content_data_valid.append(content)
    html_data += content_data_valid
    if html_data == []:
        continue
    one_item = ''
    code = ''
    for item in html_data:
        item = item.strip()
        one_item += ' '
        one_item += item
    dataset.loc[i, 'data'] = one_item
    if (i + 1) % 10 == 0:
        time_a = time_b
        time_b = time.time()
        print('{} records have been processed.time used:{:.2f}'.format(i, time_b - time_a))

1439 records have been processed.time used:5.40
1449 records have been processed.time used:18.87
1459 records have been processed.time used:14.39
1469 records have been processed.time used:31.42
1479 records have been processed.time used:8.72
1489 records have been processed.time used:15.38
1499 records have been processed.time used:15.22
time out.
1509 records have been processed.time used:71.97
1519 records have been processed.time used:24.76
time out.
1529 records have been processed.time used:35.64
1539 records have been processed.time used:25.92
time out.
1549 records have been processed.time used:39.17
time out.
1559 records have been processed.time used:9.84
1569 records have been processed.time used:16.27
time out.
1579 records have been processed.time used:35.64
1589 records have been processed.time used:25.10
time out.
1599 records have been processed.time used:42.02
time out.
1609 records have been processed.time used:16.18
1619 records have been processed.time used:15.35
16

In [10]:
dataset

,Unnamed: 0,url,target,data
0,0,http://www.zywlyy.cn/dy-xia-zai/ttdyxz10843.html,0,《兰心大剧院》投资成本高吗？份额真实有效吗？收益有保障吗？_电影天堂 ...
1,1,http://www.zywlyy.cn/dy-xia-zai/ttdyxz10778.html,0,《战疫英雄》还能投资吗？真实成本多少？投资风险大吗_电影天堂 ...
2,2,http://www.zywlyy.cn/dy-xia-zai/ttdyxz10874.html,0,《黎明之战》怎么投资出品方？收益空间大吗？个人可以参与？_电影天堂 ...
3,3,https://www.mofancy112.net/index.php/vod/detai...,1,﻿ 三街剧场在线观看 - 九月在线观看视频_影视大全_在线电影_高清影院 ...
4,4,https://www.mofancy112.net/index.php/vod/detai...,1,﻿ 不哭妈妈在线观看 - 九月在线观看视频_影视大全_在线电影_高清影院 ...
...,...,...,...,...
2032,2032,www.hdpianyuan.com,0,高清片源网_最新电影下载 高清片源网 ...
2033,2033,www.hdpianyuan.com,0,高清片源网_最新电影下载 高清片源网 ...
2034,2034,www.hdshare.cn,0,正版蓝光原盘ISO下载网站|4K蓝光电影资源|蓝光高清分享网 ...
2035,2035,www.hdsky.net,0,"HDSky高清论坛-怀旧影音,美剧,韩剧,百度云资源,交流分享社区 ..."


In [25]:
dataset

,url,target,data
0,http://www.zywlyy.cn/dy-xia-zai/ttdyxz10843.html,0,《兰心大剧院》投资成本高吗？份额真实有效吗？收益有保障吗？_电影天堂 ...
1,http://www.zywlyy.cn/dy-xia-zai/ttdyxz10778.html,0,《战疫英雄》还能投资吗？真实成本多少？投资风险大吗_电影天堂 ...
2,http://www.zywlyy.cn/dy-xia-zai/ttdyxz10874.html,0,《黎明之战》怎么投资出品方？收益空间大吗？个人可以参与？_电影天堂 ...
3,https://www.mofancy112.net/index.php/vod/detai...,1,﻿ 三街剧场在线观看 - 九月在线观看视频_影视大全_在线电影_高清影院 ...
4,https://www.mofancy112.net/index.php/vod/detai...,1,﻿ 不哭妈妈在线观看 - 九月在线观看视频_影视大全_在线电影_高清影院 ...
...,...,...,...
2032,www.hdpianyuan.com,0,高清片源网_最新电影下载 高清片源网 ...
2033,www.hdpianyuan.com,0,高清片源网_最新电影下载 高清片源网 ...
2034,www.hdshare.cn,0,正版蓝光原盘ISO下载网站|4K蓝光电影资源|蓝光高清分享网 ...
2035,www.hdsky.net,0,"HDSky高清论坛-怀旧影音,美剧,韩剧,百度云资源,交流分享社区 ..."


In [12]:
dataset.to_csv('temp.csv')

In [20]:
dataset = dataset.drop(columns=['Unnamed: 0'], axis = 1)

In [21]:
dataset.to_csv('universal.csv')

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

import jieba

# 这段代码是把上面dataframe的text部分进行分词，然后转到列表里面去，方便进行tf-idf，结果可以看下面
all_text = []
y = []
for index in dataset.index:
    if pd.isna(dataset.loc[index, 'data']):
        continue
    item_text = []
    text = dataset.loc[index, 'data']
    text = [i for i in text.split()]
    for i in text:
        for j in jieba.cut(i, cut_all=False):
            item_text.append(j)
    item_text = ' '.join(item_text)
    y.append(dataset.loc[index, 'target'])
    all_text.append(item_text)

In [34]:
MAX_FEATURES = 80

In [35]:
tfidf = TfidfVectorizer(min_df=10, max_features=MAX_FEATURES)
X_tfidf = tfidf.fit_transform(all_text)
X_labels =X_tfidf.toarray() > 0
features = tfidf.get_feature_names_out()
for index in range(len(X_labels[0])):
    if X_labels[0][index] == True:
        print(features[index])

03
04
05
06
07
08
09
10
1080p
11
12
13
15
2020
24
4k
com
hd
下载
中国
中字
动作
国语
在线
天堂
导演
影片
影视
我们
故事
更新
欧美
爱情
电影
网站
美国
观看
资源
迅雷
首页
高清


In [36]:
from sklearn.model_selection import train_test_split

X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(X_tfidf.toarray(), y, test_size=0.2)

In [37]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier


def test_models(X_train, X_test, y_train, y_test):
    
#     if method == 'tfidf':
#         X_train = tfidf.transform(X_train)
#         X_test = tfidf.transform(X_test)
#     elif method == 'count':
#         X_train = count.transform(X_train)
#         X_test = count.transform(X_test)

    model1 = MultinomialNB()
    model1.fit(X_train, y_train)
    print('Naive_bayes: {:.2f} '.format(model1.score(X_test, y_test)))



    model2 = KNeighborsClassifier()
    model2.fit(X_train, y_train)
    print('kneighbors: {:.2f}'.format(model2.score(X_test, y_test)))


    model3 = LogisticRegression()
    model3.fit(X_train, y_train)
    print('LogisticRegression: {:.2f}'.format(model3.score(X_test, y_test)))


    model4 = RandomForestClassifier()
    model4.fit(X_train, y_train)
    print('RandomForest: {:.2f}'.format(model4.score(X_test, y_test)))


    model5 = DecisionTreeClassifier()
    model5.fit(X_train, y_train)
    print('DecisionTree: {:.2f}'.format(model5.score(X_test, y_test)))


    model6 = GradientBoostingClassifier()
    model6.fit(X_train, y_train)
    print('GradientBoost: {:.2f}'.format(model6.score(X_test, y_test)))


    model7 = SVC(kernel='rbf')
    model7.fit(X_train, y_train)
    print('SVM: {:.2f}'.format(model7.score(X_test, y_test)))
    
    
    model8 = XGBClassifier()
    model8.fit(X_train, y_train)
    print('xgboost: {:.2f}'.format(model8.score(X_test, y_test)))
    
    return model1, model2, model3, model4, model5, model6, model7, model8

D:\latefallsapp\Anaconda\envs\d2l\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [38]:
model1, _, model3, model4, _, model6, model7, model8 = test_models(X_train_tf, X_test_tf, y_train_tf, y_test_tf)

Naive_bayes: 0.79 
kneighbors: 0.87
LogisticRegression: 0.86
RandomForest: 0.97
DecisionTree: 0.93
GradientBoost: 0.96
SVM: 0.91


D:\latefallsapp\Anaconda\envs\d2l\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:33:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
xgboost: 0.97


In [41]:
model8.score(X_test_tf, y_test_tf)

0.9689119170984456